In [1]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Bidirectional, LSTM, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling

# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

#file accessing
import os
# time stuff
from datetime import timedelta
import calendar


Tensorflow/Keras: 2.9.0
pandas: 1.4.2
numpy: 1.22.4
sklearn: 1.1.1
plotly: 5.9.0


In [12]:
#setting coordinates and opening files
longi="-119.349"
lat="36.1352"
lon_south=pd.read_csv('InSAR_data_south/longitude.csv')
lat_south=pd.read_csv('InSAR_data_south/latitude.csv')
CSVpath="E:/DSFellowship/InSAR_data_south/displacement/CSV/"


In [3]:
#get dates
displacement=[]
date=[]
for f in os.listdir(CSVpath):
    #print(f)
    name, ext=os.path.splitext(f)
    date.append(int(name))
    # change columns and index names into lon and lat
date.sort()

In [4]:
#get displacements
for f in date:
    z_south=pd.read_csv(CSVpath+str(f)+'.csv')
    z_south.columns=lon_south.columns
    z_south.index=lat_south.columns[:-1]
    #displacement
    displacement.append(z_south.loc[lat,longi])
print(displacement)

[0.0, -2.348, 0.29425, -1.0649, -2.7628, -3.1882, -5.0385, -6.7351, -10.232, -11.704, -12.48, -15.882, -19.311, -19.25, -18.395, -19.052, -18.866, -17.181, -20.001, -18.252, -17.879, -17.695, -19.939, -19.501, -20.068, -18.843, -21.166, -21.547, -21.667, -24.297, -24.272, -26.189, -30.104, -29.224, -31.22, -31.712, -32.181, -31.71, -34.99, -33.107, -32.633, -32.061, -33.858, -32.703, -32.967, -33.313, -33.235, -32.664, -33.158, -31.862, -32.689, -32.278, -30.776, -33.29, -33.515, -32.221, -31.883, -32.633, -33.028, -34.111, -35.192, -35.117, -35.778, -35.954, -37.331, -36.74, -34.658, -37.256, -38.246, -37.072, -38.136, -39.604, -36.93, -36.243, -37.927, -37.516, -37.033, -36.377, -36.53, -37.913, -39.063, -38.884, -37.511, -38.593, -40.715, -38.405, -40.567, -39.468, -41.736, -42.835, -44.341, -43.344, -44.155, nan, -46.06, -47.983, -49.04, -50.19, -48.78, -50.701, -49.764, -49.891, -49.823, -49.173, -49.986, -48.972, -48.171, -48.865, -49.518, -49.159]


In [5]:
#fill in empty plots
for f in range(len(displacement)):
    if(np.isnan(displacement[f])):
        if(f==0):
            displacement[f]=0
        else:
            displacement[f]=(displacement[f-1]+displacement[f+1])/2

In [6]:
#create new dataframe for one point
data_tuples=list(zip(date,displacement))
df=pd.DataFrame(data_tuples, columns=['Date','Disp'])

In [7]:
# Random Forest Regression


# Importing the datasets df.index=pd.to_datetime(df['Date'], format='%Y%m%d')
X = df.iloc[:, 0:1].values
Y = df.iloc[:, 1:2].values
#X = [pd.to_datetime(x, format='%Y%m%d') for x in X]
#print(X[0])
#print(Y)

In [8]:
#helper function to reshape data to 3d array  https://towardsdatascience.com/lstm-recurrent-neural-networks-how-to-teach-a-network-to-remember-the-past-55e54c2ff22e
def shaping(datain, timestep):
    
    # Convert input dataframe to array and flatten
    arr=datain.to_numpy().flatten() 
    
    cnt=0
    for mth in range(0, len(datain.columns)-(2*timestep)+1): # Define range 
        cnt=cnt+1 # Gives us the number of samples. Later used to reshape the data
        X_start=mth # Start month for inputs of each sample
        X_end=mth+timestep # End month for inputs of each sample
        Y_start=mth+timestep # Start month for targets of each sample. Note, start is inclusive and end is exclusive, that's why X_end and Y_start is the same number
        Y_end=mth+2*timestep # End month for targets of each sample.  
        
        # Assemble input and target arrays containing all samples
        if mth==0:
            X_comb=arr[X_start:X_end]
            Y_comb=arr[Y_start:Y_end]
        else: 
            X_comb=np.append(X_comb, arr[X_start:X_end])
            Y_comb=np.append(Y_comb, arr[Y_start:Y_end])
    
    # Reshape input and target arrays
    X_out=np.reshape(X_comb, (cnt, timestep, 1))
    Y_out=np.reshape(Y_comb, (cnt, timestep, 1))
    return X_out, Y_out

In [9]:
##### Step 1 - Specify parameters
timestep=15 #3 months?
longi="-119.349"
lat="36.1352"



In [13]:
df

,Date,Disp
0,20141108,0.00000
1,20141202,-2.34800
2,20141226,0.29425
3,20150212,-1.06490
4,20150308,-2.76280
...,...,...
105,20181205,-48.97200
106,20181217,-48.17100
107,20181229,-48.86500
108,20190110,-49.51800


In [15]:
df_train=df.iloc[:80, :].copy()
df_test=df.iloc[80:, :].copy()
print(df_train)


        Date      Disp
0   20141108   0.00000
1   20141202  -2.34800
2   20141226   0.29425
3   20150212  -1.06490
4   20150308  -2.76280
..       ...       ...
75  20171222 -37.51600
76  20180103 -37.03300
77  20180115 -36.37700
78  20180127 -36.53000
79  20180208 -37.91300

[80 rows x 2 columns]


In [17]:
##### Step 2 - Prepare data

# Split data into train and test dataframes
df_train=df.iloc[:80, :].copy()
df_test=df.iloc[80:, :].copy()

# Select one location
#dfloc_train = df_train[df_train.index==location].copy()
#dfloc_test = df_test[df_test.index==location].copy()

# Use previously defined shaping function to reshape the data for LSTM
X_train, Y_train = shaping(datain=df_train, timestep=timestep)
X_test, Y_test = shaping(datain=df_test, timestep=timestep)




UnboundLocalError: local variable 'X_comb' referenced before assignment

In [ ]:
##### Step 3 - Specify the structure of a Neural Network
model = Sequential(name="LSTM-Model") # Model
model.add(Input(shape=(X_train.shape[1],X_train.shape[2]), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model.add(Bidirectional(LSTM(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False), name='Hidden-LSTM-Encoder-Layer')) # Encoder Layer
model.add(RepeatVector(Y_train.shape[1], name='Repeat-Vector-Layer')) # Repeat Vector
model.add(Bidirectional(LSTM(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False, return_sequences=True), name='Hidden-LSTM-Decoder-Layer')) # Decoder Layer
model.add(TimeDistributed(Dense(units=1, activation='linear'), name='Output-Layer')) # Output Layer, Linear(x) = x


##### Step 4 - Compile the model
model.compile(optimizer='adam', # default='rmsprop', an algorithm to be used in backpropagation
              loss='mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics=['MeanSquaredError', 'MeanAbsoluteError'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. 
              loss_weights=None, # default=None, Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs.
              weighted_metrics=None, # default=None, List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
              run_eagerly=None, # Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function.
              steps_per_execution=None # Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead.
             )


##### Step 5 - Fit the model on the dataset
history = model.fit(X_train, # input data
                    Y_train, # target data
                    batch_size=1, # Number of samples per gradient update. If unspecified, batch_size will default to 32.
                    epochs=1000, # default=1, Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
                    verbose=0, # default='auto', ('auto', 0, 1, or 2). Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy.
                    callbacks=None, # default=None, list of callbacks to apply during training. See tf.keras.callbacks
                    validation_split=0.2, # default=0.0, Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. 
                    #validation_data=(X_test, y_test), # default=None, Data on which to evaluate the loss and any model metrics at the end of each epoch. 
                    shuffle=True, # default=True, Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
                    class_weight=None, # default=None, Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
                    sample_weight=None, # default=None, Optional Numpy array of weights for the training samples, used for weighting the loss function (during training only).
                    initial_epoch=0, # Integer, default=0, Epoch at which to start training (useful for resuming a previous training run).
                    steps_per_epoch=None, # Integer or None, default=None, Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. When training with input tensors such as TensorFlow data tensors, the default None is equal to the number of samples in your dataset divided by the batch size, or 1 if that cannot be determined. 
                    validation_steps=None, # Only relevant if validation_data is provided and is a tf.data dataset. Total number of steps (batches of samples) to draw before stopping when performing validation at the end of every epoch.
                    validation_batch_size=None, # Integer or None, default=None, Number of samples per validation batch. If unspecified, will default to batch_size.
                    validation_freq=100, # default=1, Only relevant if validation data is provided. If an integer, specifies how many training epochs to run before a new validation run is performed, e.g. validation_freq=2 runs validation every 2 epochs.
                    max_queue_size=10, # default=10, Used for generator or keras.utils.Sequence input only. Maximum size for the generator queue. If unspecified, max_queue_size will default to 10.
                    workers=1, # default=1, Used for generator or keras.utils.Sequence input only. Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
                    use_multiprocessing=True, # default=False, Used for generator or keras.utils.Sequence input only. If True, use process-based threading. If unspecified, use_multiprocessing will default to False. 
                   )


##### Step 6 - Use model to make predictions
# Predict results on training data
pred_train = model.predict(X_train)
# Predict results on test data
pred_test = model.predict(X_test)


##### Step 7 - Print Performance Summary
print("")
print('-------------------- Model Summary --------------------')
model.summary() # print model summary
print("")
print('-------------------- Weights and Biases --------------------')
print("Too many parameters to print but you can use the code provided if needed")
print("")
#for layer in model.layers:
#    print(layer.name)
#    for item in layer.get_weights():
#        print("  ", item)
#print("")

# Print the last value in the evaluation metrics contained within history file
print('-------------------- Evaluation on Training Data --------------------')
for item in history.history:
    print("Final", item, ":", history.history[item][-1])
print("")

# Evaluate the model on the test data using "evaluate"
print('-------------------- Evaluation on Test Data --------------------')
results = model.evaluate(X_test, Y_test)
print("")